<a href="https://colab.research.google.com/github/ysmnpksy/Final-Project/blob/main/Set1_BagOfWords_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Final Project Prototype: Deep Learning Algorithm to Recognize Spoilers 

In order to show proof of concept, I have developed a deep learning bag-of-words algorithm as my prototype. Deep learning allows for pattern recognition in words, sentences, and paragraphs, meaning it can be applied to this project in order to classify whether a piece of text includes a spoiler or not. 

My development process can be examined through this notebook. I have included explanations and justification in text cells in-between the code to show my understand and demonstrate how I will continue the development phase moving forward.

# Part 1: Kaggle & Dataset
As the first step, I am preparing the notebook by installing Kaggle and then the dataset.  

## 1.1: Installing Kaggle 
I have installed Kaggle using the following commands. While the output shows "already satisfied", thanks to Google Colab, I am leaving the code here for good practice, as it could be needed when running this notebook on a different platform. 

I have generated an API login using my Kaggle account and uploaded the JSON file, `kaggle.json`, containing the username and key, to this notebook. If running this notebook, and not viewing via the HTML format, the API key, included in the submission folder, needs to be uploaded to the notebook before running the first code cell.

In [ ]:
# making kaggle directory 
! mkdir ~/.kaggle

# copying api login info into directory 
! cp kaggle.json ~/.kaggle/

# allocating required permissions 
! chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


## 1.2: Installing Dataset 

The [dataset](www.kaggle.com/rmisra/imdb-spoiler-dataset) used for this prototype and any further iterations of this model is the IMDB Spoiler Dataset, downloaded from Kaggle. This dataset was developed using IMDB and contains two different databases: movies and reviews. The reviews dataset, which will be the one used for this model, contains 573,913 records. 

IMDB allows users to know which reviews contain spoilers by tagging the reviews with "Warning: Spoilers". Reviews which include spoilers but have not been indicated to do so by the reviewer will be removed when reported under the "Spoiler without warning" option. Due to this, I can be confident that this dataset includes accurate information, making this dataset was the best option for the project; the only downside is that the reviews are only taken from movies, and not a mix of movies and TV shows, which would have been more applicable to this project. 

In the code cell below, I am downloading the dataset from Kaggle and unzipping it to make the data accessible.


In [ ]:
! kaggle datasets download rmisra/imdb-spoiler-dataset

# unzipping dataset 
! unzip imdb-spoiler-dataset.zip

 97% 323M/331M [00:10<00:00, 37.3MB/s]
100% 331M/331M [00:10<00:00, 32.5MB/s]
Archive:  imdb-spoiler-dataset.zip
  inflating: IMDB_movie_details.json  
  inflating: IMDB_reviews.json       


### 1.2.1: Reviews Database 

As mentioned above, the database I will be using from this dataset is the reviews database, which has over five hundred thousand reviews. This database contains information about each review, such as the date it was made, the movie ID of the movie it was about, the review text itself and a boolean field which indicates whether or not it is a spoiler. 

The code cell below shows information about this database and a sample consisting of the first 5 reviews. As shown, this dataset consists of 573,913 records, 150,924 of them which contain spoilers. This is a significant number of records and should be enough to train this algorithm without overfitting issues.

In [ ]:
# importing pandas to read the JSON files
import pandas as pd 

# information regarding reviews file
all_reviews = pd.read_json('../content/IMDB_reviews.json', lines=True)

print('Total number of reviews:', all_reviews['review_date'].count())
print('Total number of reviews that contain spoilers:', all_reviews['is_spoiler'].sum())
print('User reviews shape:', all_reviews.shape)
print()

print('First 5 user reviews:')
all_reviews.head()

Total number of reviews: 573913
Total number of reviews that contain spoilers: 150924
User reviews shape: (573913, 7)

First 5 user reviews:


,review_date,movie_id,user_id,is_spoiler,review_text,rating,review_summary
0,10 February 2006,tt0111161,ur1898687,True,"In its Oscar year, Shawshank Redemption (writt...",10,A classic piece of unforgettable film-making.
1,6 September 2000,tt0111161,ur0842118,True,The Shawshank Redemption is without a doubt on...,10,Simply amazing. The best film of the 90's.
2,3 August 2001,tt0111161,ur1285640,True,I believe that this film is the best story eve...,8,The best story ever told on film
3,1 September 2002,tt0111161,ur1003471,True,"**Yes, there are SPOILERS here**This film has ...",10,Busy dying or busy living?
4,20 May 2004,tt0111161,ur0226855,True,At the heart of this extraordinary movie is a ...,8,"Great story, wondrously told and acted"


In [ ]:
all_reviews.iloc[0].review_text

'In its Oscar year, Shawshank Redemption (written and directed by Frank Darabont, after the novella Rita Hayworth and the Shawshank Redemption, by Stephen King) was nominated for seven Academy Awards, and walked away with zero. Best Picture went to Forrest Gump, while Shawshank and Pulp Fiction were "just happy to be nominated." Of course hindsight is 20/20, but while history looks back on Gump as a good film, Pulp and Redemption are remembered as some of the all-time best. Pulp, however, was a success from the word "go," making a huge splash at Cannes and making its writer-director an American master after only two films. For Andy Dufresne and Co., success didn\'t come easy. Fortunately, failure wasn\'t a life sentence.After opening on 33 screens with take of $727,327, the $25M film fell fast from theatres and finished with a mere $28.3M. The reasons for failure are many. Firstly, the title is a clunker. While iconic to fans today, in 1994, people knew not and cared not what a \'Shaws

### 1.2.2: Movies Database 
The second database included in this dataset is the movies database. This database contains all the information regarding each movie itself, such as the genre, duration, and release date. Information about this database and a sample of the first 5 movies can be seen below.

In [ ]:
# information regarding movie details file 
all_movies = pd.read_json('../content/IMDB_movie_details.json', lines=True)

print('Total number of movies:', all_movies['movie_id'].count())
print('Movie details shape:', all_movies.shape)
print()

print('First 5 movie details:')

all_movies.head()

Total number of movies: 1572
Movie details shape: (1572, 7)

First 5 movie details:


,movie_id,plot_summary,duration,genre,rating,release_date,plot_synopsis
0,tt0105112,"Former CIA analyst, Jack Ryan is in England wi...",1h 57min,"[Action, Thriller]",6.9,1992-06-05,"Jack Ryan (Ford) is on a ""working vacation"" in..."
1,tt1204975,"Billy (Michael Douglas), Paddy (Robert De Niro...",1h 45min,[Comedy],6.6,2013-11-01,Four boys around the age of 10 are friends in ...
2,tt0243655,"The setting is Camp Firewood, the year 1981. I...",1h 37min,"[Comedy, Romance]",6.7,2002-04-11,
3,tt0040897,"Fred C. Dobbs and Bob Curtin, both down on the...",2h 6min,"[Adventure, Drama, Western]",8.3,1948-01-24,Fred Dobbs (Humphrey Bogart) and Bob Curtin (T...
4,tt0126886,Tracy Flick is running unopposed for this year...,1h 43min,"[Comedy, Drama, Romance]",7.3,1999-05-07,Jim McAllister (Matthew Broderick) is a much-a...


# Part 2: Processing the data 

The data needs to be processed before a model can be built. Here I will be loading the reviews and the spoiler labels into lists tokenizing using one-hot encoding, and splitting the dataset into training, validation and testing sets.


## 2.1: Loading the data
Here the labels are being added to a `labels` list while the review text is being added to a `texts` list.


In [ ]:
import json

labels = [] 
texts = []

with open('IMDB_reviews.json', 'r') as json_file:
  for jsonObj in json_file:
    data = json.loads(jsonObj)
    if data['is_spoiler'] == True:
       labels.append(1)
    else:
       labels.append(0)
    texts.append(data['review_text'])

## 2.2: Tokenizing the text
Before splitting, I am tokenizing the text data using one-hot encoding. To do this I am using the `Tokenizer` module from Keras.

I have cut the reviews of after a maximum of 500 words and am only considering the most frequent 15000 words in the dataset. 

After tokenizing, I have vectorized the data using the `pad_sequences` module from Keras, which converts the list of tokenized data, integers, into 2D tensors, which can then be fed into the neural network. I have also vectorized the list of labels by converting it to a list of floating point numbers.

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

maxlen = 500 # cuts reviews off after 500 words 
max_words = 15000 # considers only the top 15000 words in the dataset 

# tokenizing texts 
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

# vectorizing texts
data = pad_sequences(sequences, maxlen=maxlen)

# vectorizing labels 
labels = np.asarray(labels).astype('float32')

print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Found 379856 unique tokens.
Shape of data tensor: (573913, 500)
Shape of label tensor: (573913,)


## 2.3: Splitting Data
Initially, I am splitting the dataset into training and testing sets. This way I will be testing the model on unseen data only, ensuring information leaking does not affect the final results. 

I am first shuffling the data, since the JSON file lists all reviews including spoilers first, then splitting it. I will be training on 200,000 samples and testing using 100.000 for this prototype.  


In [ ]:
# suffling data
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

# setting number of training and testing samples
training_samples = 200000
testing_samples = 100000

# splitting into training and testing sets
x_train = data[:training_samples]
y_train = labels[:training_samples]
x_test = data[training_samples: training_samples + testing_samples]
y_test = labels[training_samples: training_samples + testing_samples]

In [ ]:
print('Training data shape:',x_train.shape)
print('Training labels shape:', y_train.shape)

print('Test data shape:',x_test.shape)
print('Test labels shape:',y_test.shape)

Training data shape: (200000, 500)
Training labels shape: (200000,)
Test data shape: (100000, 500)
Test labels shape: (100000,)


I am then further splitting the training set into a partial training and validation set. This way the data used to validate will not be the same as that used to train, which could cause the model to overfit. I will be training on 150,000 samples and validating on 50,000. 

In [ ]:
# setting number of training and validation samples
partial_training_samples = 150000
validation_samples = 50000

# splitting into training and validation sets
x_partial_train = data[:partial_training_samples]
y_partial_train = labels[:partial_training_samples]
x_val = data[partial_training_samples: partial_training_samples + validation_samples]
y_val = labels[partial_training_samples: partial_training_samples + validation_samples]

In [ ]:
print('Partial training data shape:',x_partial_train.shape)
print('Partial training labels shape:', y_partial_train.shape)

print('Validation data shape:',x_val.shape)
print('Validation labels shape:',y_val.shape)

Partial training data shape: (150000, 500)
Partial training labels shape: (150000,)
Validation data shape: (50000, 500)
Validation labels shape: (50000,)


# Part 3: The Initial Model

Here I am developing the initial model, which I will then tune.

In [ ]:
import keras.backend as K
from sklearn.metrics import f1_score as f1

def f1_score(y_true, y_pred):
  tp = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
  posp = K.sum(K.round(K.clip(y_pred, 0, 1)))
  predp = K.sum(K.round(K.clip(y_true, 0, 1)))

  precision = tp / (posp + K.epsilon())
  recall = tp / (predp + K.epsilon())

  f1_score = 2 * (precision * recall) / (precision + recall + K.epsilon())
  return f1_score 

## 3.1: Defining Initial Model 

The initial model I've defined can be seen below. This model consists of 6 layers. 

The first two layers are used for word embeddings, which is used to map human language into a geometric space. The `Embedding` layer takes integers as input, finds the integer in an internal dictionary, and returns corresponding word vectors, allowing for vectors to be associated with words. 

The next four layers are classification layers, the last being an output layer.  


In [ ]:
import tensorflow as tf
from tensorflow.keras import models
from tensorflow.keras import layers
from keras.layers import Embedding, Flatten

model = models.Sequential()

# Embedding layers 
# network will learn 8-dimensional embeddings for each of the 15,000 words
model.add(Embedding(15000, 8, input_length=maxlen))
# flattens the 3D tensor output to a 2D tensor
model.add(Flatten())

# training single dense layer for classfication 
model.add(layers.Dense(128, activation = 'relu'))
model.add(layers.Dense(64, activation = 'relu'))
model.add(layers.Dense(32, activation = 'relu'))

# output layer 
model.add(layers.Dense(1, activation = 'sigmoid'))

model.compile(optimizer = 'rmsprop', 
              loss = 'binary_crossentropy', 
              metrics = ['accuracy', f1_score])

model.build()
model.summary()

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_13 (Embedding)    (None, 500, 8)            120000    
                                                                 
 flatten_13 (Flatten)        (None, 4000)              0         
                                                                 
 dense_50 (Dense)            (None, 128)               512128    
                                                                 
 dense_51 (Dense)            (None, 64)                8256      
                                                                 
 dense_52 (Dense)            (None, 32)                2080      
                                                                 
 dense_53 (Dense)            (None, 1)                 33        
                                                                 
Total params: 642,497
Trainable params: 642,497
Non-t

## 3.2: Training Model 1

Here I have trained the initial model using the partial training data and validated it on the validation data. I have trained for 10 epochs with a batch size of 32. 

In [ ]:
history = model.fit(x_partial_train, y_partial_train,
                    epochs = 10,
                    batch_size = 32,
                    validation_data = (x_val, y_val))

Epoch 1/10
4688/4688 [==============================] - 56s 12ms/step - loss: 0.5086 - accuracy: 0.7601 - f1_score: 0.2948 - val_loss: 0.4994 - val_accuracy: 0.7647 - val_f1_score: 0.3825
Epoch 2/10
4688/4688 [==============================] - 54s 11ms/step - loss: 0.4804 - accuracy: 0.7818 - f1_score: 0.4153 - val_loss: 0.5406 - val_accuracy: 0.7593 - val_f1_score: 0.4063
Epoch 3/10
4688/4688 [==============================] - 56s 12ms/step - loss: 0.4441 - accuracy: 0.8078 - f1_score: 0.5089 - val_loss: 0.5623 - val_accuracy: 0.7594 - val_f1_score: 0.3720
Epoch 4/10
4688/4688 [==============================] - 56s 12ms/step - loss: 0.4041 - accuracy: 0.8310 - f1_score: 0.5840 - val_loss: 0.6071 - val_accuracy: 0.7582 - val_f1_score: 0.2972
Epoch 5/10
4688/4688 [==============================] - 56s 12ms/step - loss: 0.3671 - accuracy: 0.8493 - f1_score: 0.6415 - val_loss: 0.7006 - val_accuracy: 0.7376 - val_f1_score: 0.4309
Epoch 6/10
4688/4688 [==============================] - 60s 

In [ ]:
history_dict = history.history
history_dict.keys()

# Part 4: Tuning & Regualizing
Here I am tuning the learning rate and adding regularization to achieve the highest possible accuracy.  


## 4.1: Learning Rate

I am tuning the learning rate of the optimiser. The learning rate determines how much the weights of the network are updated according to the accuracy achieved. If the learning rate is too small, the model may never find the optimal set of weights as they will be updated in very small amounts. If it is too large, the optimal set of weights could get skipped over during training. 

As I will be building many iterations of this model, I have made a `build_model` and `train_model` function to make this process more efficient, shown below. 

In [ ]:
from tensorflow.keras import optimizers

def build_model(learning_rate):
  model = models.Sequential()
  
  # Embedding layers
  model.add(Embedding(15000, 8, input_length=maxlen))
  # flattens the 3D tensor output to a 2D tensor
  model.add(Flatten())
  
  # training single dense layer for classfication 
  model.add(layers.Dense(128, activation = 'relu'))
  model.add(layers.Dense(64, activation = 'relu'))
  model.add(layers.Dense(32, activation = 'relu'))  
  
  # outup layer 
  model.add(layers.Dense(1, activation = 'sigmoid'))
  
  model.compile(optimizer=optimizers.RMSprop(learning_rate=learning_rate),
              loss = 'binary_crossentropy', 
              metrics = ['accuracy', f1_score])
  
  return model
  
def train_model(learning_rate): 
  model = build_model(learning_rate)
  history = model.fit(x_partial_train, y_partial_train,
                    epochs = 10,
                    batch_size = 32,
                    validation_data = (x_val, y_val))
  
  return history.history

### Model 2: Learning Rate 0.005

In the initial model, I had the learning rate set to the default. In model 1, I am increasing it to 0.005 to see if this will help achieve a higher accuracy on the validation set. As shown below, this model starts degrading straight away, showing that this learning rate is too high.

In [ ]:
model_2 = train_model(0.005)

Epoch 1/10
4688/4688 [==============================] - 69s 15ms/step - loss: 0.5171 - accuracy: 0.7565 - f1_score: 0.2520 - val_loss: 0.5065 - val_accuracy: 0.7555 - val_f1_score: 0.3966
Epoch 2/10
4688/4688 [==============================] - 69s 15ms/step - loss: 0.5029 - accuracy: 0.7710 - f1_score: 0.3859 - val_loss: 0.5681 - val_accuracy: 0.7506 - val_f1_score: 0.4332
Epoch 3/10
4688/4688 [==============================] - 68s 15ms/step - loss: 0.4851 - accuracy: 0.7843 - f1_score: 0.4191 - val_loss: 0.5586 - val_accuracy: 0.7607 - val_f1_score: 0.3811
Epoch 4/10
4688/4688 [==============================] - 67s 14ms/step - loss: 0.4621 - accuracy: 0.8029 - f1_score: 0.4765 - val_loss: 0.5999 - val_accuracy: 0.7605 - val_f1_score: 0.3609
Epoch 5/10
4688/4688 [==============================] - 68s 14ms/step - loss: 0.4302 - accuracy: 0.8208 - f1_score: 0.5387 - val_loss: 1.2113 - val_accuracy: 0.6669 - val_f1_score: 0.4992
Epoch 6/10
4688/4688 [==============================] - 68s 

### Model 3: Learning Rate 0.003

In this next model, I am decreasing the learning rate to 0.003. Again, the validation accuracy starts degrading straight away, showing this learning rate is also too high for this model. 

In [ ]:
model_2 = train_model(0.003)

Epoch 1/10
4688/4688 [==============================] - 67s 14ms/step - loss: 0.5108 - accuracy: 0.7593 - f1_score: 0.2966 - val_loss: 0.5122 - val_accuracy: 0.7629 - val_f1_score: 0.4028
Epoch 2/10
4688/4688 [==============================] - 68s 14ms/step - loss: 0.4766 - accuracy: 0.7832 - f1_score: 0.4347 - val_loss: 0.5046 - val_accuracy: 0.7617 - val_f1_score: 0.3455
Epoch 3/10
4688/4688 [==============================] - 68s 14ms/step - loss: 0.4302 - accuracy: 0.8139 - f1_score: 0.5472 - val_loss: 0.6306 - val_accuracy: 0.7368 - val_f1_score: 0.4465
Epoch 4/10
4688/4688 [==============================] - 68s 15ms/step - loss: 0.3810 - accuracy: 0.8425 - f1_score: 0.6305 - val_loss: 1.3153 - val_accuracy: 0.6559 - val_f1_score: 0.4744
Epoch 5/10
4688/4688 [==============================] - 75s 16ms/step - loss: 0.3389 - accuracy: 0.8617 - f1_score: 0.6801 - val_loss: 0.8262 - val_accuracy: 0.7521 - val_f1_score: 0.3262
Epoch 6/10
4688/4688 [==============================] - 68s 

### Model 4: Learning Rate 0.0005

In the next iteration I am decreasing the learning rate to 0.005 to see if this will improve on the validation accuracy compared to the initial model. 

In [ ]:
model_4 = train_model(0.0005)

Epoch 1/10
4688/4688 [==============================] - 67s 14ms/step - loss: 0.5061 - accuracy: 0.7607 - f1_score: 0.2911 - val_loss: 0.5091 - val_accuracy: 0.7583 - val_f1_score: 0.4252
Epoch 2/10
4688/4688 [==============================] - 67s 14ms/step - loss: 0.4725 - accuracy: 0.7840 - f1_score: 0.4221 - val_loss: 0.5019 - val_accuracy: 0.7617 - val_f1_score: 0.2560
Epoch 3/10
4688/4688 [==============================] - 67s 14ms/step - loss: 0.4359 - accuracy: 0.8088 - f1_score: 0.5123 - val_loss: 0.5634 - val_accuracy: 0.7534 - val_f1_score: 0.4256
Epoch 4/10
4688/4688 [==============================] - 69s 15ms/step - loss: 0.3969 - accuracy: 0.8313 - f1_score: 0.5810 - val_loss: 0.6434 - val_accuracy: 0.7507 - val_f1_score: 0.3924
Epoch 5/10
4688/4688 [==============================] - 71s 15ms/step - loss: 0.3615 - accuracy: 0.8504 - f1_score: 0.6369 - val_loss: 0.8110 - val_accuracy: 0.7558 - val_f1_score: 0.3259
Epoch 6/10
4688/4688 [==============================] - 69s 

## 4.2: Weight Regularization

I am adding weight regularization, where the network is forced to set weights to small values, making the distribution of them more regular. This is done by adding a cost to the loss function for larger weights. There are two types of costs:

1. L1 regularization: This is where the cost added to the weight is equivalent to the absolute value of the weight coefficients.
2. L2 regularization: This is where the cost added to the weight is equivalent to the square of the weight coefficients.

These two types of costs can be added to a network individually or simultaneously. I will be testing all three methods to find the one that works best for this model. In order to implement regularizers I have imported the regularizers module from Keras, shown below. I have also made a new build function to train the models. 

In [ ]:
from keras import regularizers

def train_reg_model(model): 
  history = model.fit(x_partial_train, y_partial_train,
                    epochs = 10,
                    batch_size = 32,
                    validation_data = (x_val, y_val))
  
  return history.history

### Model 5: L1 Regularization 

The first model I am training includes L1 regularization. This model peaks at epoch 5, to 0.762, which is not higher than the initial model.

In [ ]:
model = models.Sequential()
model.add(Embedding(15000, 8, input_length=maxlen))
model.add(Flatten())
model.add(layers.Dense(128, kernel_regularizer=regularizers.l1(0.001), activation = 'relu'))
model.add(layers.Dense(64, kernel_regularizer=regularizers.l1(0.001), activation = 'relu'))
model.add(layers.Dense(32, kernel_regularizer=regularizers.l1(0.001),activation = 'relu'))


model.add(layers.Dense(1, activation = 'sigmoid'))

model.compile(optimizer = 'rmsprop', 
              loss = 'binary_crossentropy', 
              metrics = ['accuracy', f1_score])

model.build()

model_4 = train_reg_model(model)

Epoch 1/10
4688/4688 [==============================] - 79s 17ms/step - loss: 0.8421 - accuracy: 0.7511 - f1_score: 0.1532 - val_loss: 0.8068 - val_accuracy: 0.7571 - val_f1_score: 0.2605
Epoch 2/10
4688/4688 [==============================] - 77s 16ms/step - loss: 0.7980 - accuracy: 0.7621 - f1_score: 0.3044 - val_loss: 0.8068 - val_accuracy: 0.7583 - val_f1_score: 0.3582
Epoch 3/10
4688/4688 [==============================] - 82s 18ms/step - loss: 0.7927 - accuracy: 0.7642 - f1_score: 0.3268 - val_loss: 0.8050 - val_accuracy: 0.7593 - val_f1_score: 0.2793
Epoch 4/10
4688/4688 [==============================] - 82s 18ms/step - loss: 0.7892 - accuracy: 0.7653 - f1_score: 0.3430 - val_loss: 0.7946 - val_accuracy: 0.7615 - val_f1_score: 0.3175
Epoch 5/10
4688/4688 [==============================] - 77s 17ms/step - loss: 0.7874 - accuracy: 0.7662 - f1_score: 0.3498 - val_loss: 0.7954 - val_accuracy: 0.7617 - val_f1_score: 0.2948
Epoch 6/10
4688/4688 [==============================] - 83s 

### Model 6: L2 Regulaization 

The second model I am training implements L2 regularization. This model peaks at epoch 3, to 0.766.

In [ ]:
model = models.Sequential()
model.add(Embedding(15000, 8, input_length=maxlen))
model.add(Flatten())
model.add(layers.Dense(128, kernel_regularizer=regularizers.l2(0.001), activation = 'relu'))
model.add(layers.Dense(64, kernel_regularizer=regularizers.l2(0.001), activation = 'relu'))
model.add(layers.Dense(32, kernel_regularizer=regularizers.l2(0.001),activation = 'relu'))


model.add(layers.Dense(1, activation = 'sigmoid'))

model.compile(optimizer = 'rmsprop', 
              loss = 'binary_crossentropy', 
              metrics = ['accuracy', f1_score])

model.build()

model_5 = train_reg_model(model)

Epoch 1/10
4688/4688 [==============================] - 80s 17ms/step - loss: 0.5272 - accuracy: 0.7585 - f1_score: 0.2761 - val_loss: 0.5149 - val_accuracy: 0.7596 - val_f1_score: 0.2942
Epoch 2/10
4688/4688 [==============================] - 78s 17ms/step - loss: 0.5080 - accuracy: 0.7667 - f1_score: 0.3469 - val_loss: 0.5151 - val_accuracy: 0.7616 - val_f1_score: 0.3443
Epoch 3/10
4688/4688 [==============================] - 78s 17ms/step - loss: 0.4991 - accuracy: 0.7697 - f1_score: 0.3694 - val_loss: 0.5114 - val_accuracy: 0.7624 - val_f1_score: 0.3098
Epoch 4/10
4688/4688 [==============================] - 84s 18ms/step - loss: 0.4911 - accuracy: 0.7760 - f1_score: 0.3994 - val_loss: 0.5172 - val_accuracy: 0.7598 - val_f1_score: 0.4066
Epoch 5/10
4688/4688 [==============================] - 78s 17ms/step - loss: 0.4827 - accuracy: 0.7829 - f1_score: 0.4331 - val_loss: 0.5204 - val_accuracy: 0.7606 - val_f1_score: 0.3652
Epoch 6/10
4688/4688 [==============================] - 78s 

### Model 7: L1 & L2 Regulaization 

The last model I am implementing includes L1 and L2 regularization simultaneously. This model achieved a highest validation accuracy of 0.765 at epoch 8. 

In [ ]:
model = models.Sequential()
model.add(Embedding(15000, 8, input_length=maxlen))
model.add(Flatten())
model.add(layers.Dense(128, kernel_regularizer=regularizers.l1_l2(l1=0.001, l2=0.001), activation = 'relu'))
model.add(layers.Dense(64, kernel_regularizer=regularizers.l1_l2(l1=0.001, l2=0.001), activation = 'relu'))
model.add(layers.Dense(32, kernel_regularizer=regularizers.l1_l2(l1=0.001, l2=0.001),activation = 'relu'))


model.add(layers.Dense(1, activation = 'sigmoid'))

model.compile(optimizer = 'rmsprop', 
              loss = 'binary_crossentropy', 
              metrics = ['accuracy', f1_score])

model.build()

model_6 = train_reg_model(model)

Epoch 1/10
4688/4688 [==============================] - 92s 19ms/step - loss: 0.8503 - accuracy: 0.7492 - f1_score: 0.1321 - val_loss: 0.8090 - val_accuracy: 0.7576 - val_f1_score: 0.3208
Epoch 2/10
4688/4688 [==============================] - 90s 19ms/step - loss: 0.7976 - accuracy: 0.7618 - f1_score: 0.2982 - val_loss: 0.8022 - val_accuracy: 0.7570 - val_f1_score: 0.3567
Epoch 3/10
4688/4688 [==============================] - 91s 20ms/step - loss: 0.7923 - accuracy: 0.7643 - f1_score: 0.3286 - val_loss: 0.8045 - val_accuracy: 0.7583 - val_f1_score: 0.2241
Epoch 4/10
4688/4688 [==============================] - 90s 19ms/step - loss: 0.7899 - accuracy: 0.7651 - f1_score: 0.3416 - val_loss: 0.7969 - val_accuracy: 0.7615 - val_f1_score: 0.3508
Epoch 5/10
4688/4688 [==============================] - 92s 20ms/step - loss: 0.7880 - accuracy: 0.7659 - f1_score: 0.3465 - val_loss: 0.8198 - val_accuracy: 0.7440 - val_f1_score: 0.4511
Epoch 6/10
4688/4688 [==============================] - 87s 

## 4.4: Dropout 

Dropout is another regularizing technique which I will be implementing. It drops, i.e., sets to zero, a number of output features of a model during training, breaking apart coincidental patterns that aren’t significant, allowing the final model to be more generalisable. I have remade the build_model and train_model functions, shown below. 

In [ ]:
def build_drop_model(dropout):
  model = models.Sequential()
  model.add(Embedding(15000, 8, input_length=maxlen))
  model.add(Flatten())
  model.add(layers.Dense(128, activation = 'relu'))
  model.add(layers.Dropout(dropout)) # dropout layer
  model.add(layers.Dense(64, activation = 'relu'))
  model.add(layers.Dropout(dropout)) # dropout layer
  model.add(layers.Dense(32, kernel_regularizer=regularizers.l2(0.001),activation = 'relu'))
  model.add(layers.Dense(1, activation = 'sigmoid'))
  
  model.compile(optimizer = 'rmsprop', 
              loss = 'binary_crossentropy', 
              metrics = ['accuracy', f1_score])
  
  return model

def train_drop_model(dropout): 
  model = build_drop_model(dropout)
  history = model.fit(x_partial_train, y_partial_train,
                    epochs = 10,
                    batch_size = 32,
                    validation_data = (x_val, y_val))
  
  return history.history

### Model 8: Dropout 30%
I am stating by adding a dropout rate of 30% to two layers in the model. This has not significantly changed the highest validation accuracy, which is still at 0.765. This could be because the dropout rate was either too high or too low. 

In [ ]:
model_7 = train_drop_model(0.3)

Epoch 1/10
4688/4688 [==============================] - 57s 12ms/step - loss: 0.5214 - accuracy: 0.7582 - f1_score: 0.2798 - val_loss: 0.5044 - val_accuracy: 0.7670 - val_f1_score: 0.3514
Epoch 2/10
4688/4688 [==============================] - 57s 12ms/step - loss: 0.4972 - accuracy: 0.7730 - f1_score: 0.3741 - val_loss: 0.5244 - val_accuracy: 0.7684 - val_f1_score: 0.3595
Epoch 3/10
4688/4688 [==============================] - 57s 12ms/step - loss: 0.4765 - accuracy: 0.7875 - f1_score: 0.4425 - val_loss: 0.5040 - val_accuracy: 0.7677 - val_f1_score: 0.3548
Epoch 4/10
4688/4688 [==============================] - 55s 12ms/step - loss: 0.4476 - accuracy: 0.8057 - f1_score: 0.5075 - val_loss: 0.5224 - val_accuracy: 0.7579 - val_f1_score: 0.4393
Epoch 5/10
4688/4688 [==============================] - 56s 12ms/step - loss: 0.4206 - accuracy: 0.8224 - f1_score: 0.5622 - val_loss: 0.5222 - val_accuracy: 0.7628 - val_f1_score: 0.3680
Epoch 6/10
4688/4688 [==============================] - 58s 

### Model 9: Dropout 20%

I have dropped the dropout rate to 20% for the next model. This has increased the highest validation accuracy to 0.767 at epoch 2. 

In [ ]:
model_8 = train_drop_model(0.2)

Epoch 1/10
4688/4688 [==============================] - 56s 12ms/step - loss: 0.5173 - accuracy: 0.7595 - f1_score: 0.2875 - val_loss: 0.5091 - val_accuracy: 0.7654 - val_f1_score: 0.3619
Epoch 2/10
4688/4688 [==============================] - 56s 12ms/step - loss: 0.4920 - accuracy: 0.7752 - f1_score: 0.3926 - val_loss: 0.5139 - val_accuracy: 0.7667 - val_f1_score: 0.3315
Epoch 3/10
4688/4688 [==============================] - 56s 12ms/step - loss: 0.4686 - accuracy: 0.7921 - f1_score: 0.4676 - val_loss: 0.5062 - val_accuracy: 0.7662 - val_f1_score: 0.3996
Epoch 4/10
4688/4688 [==============================] - 55s 12ms/step - loss: 0.4346 - accuracy: 0.8129 - f1_score: 0.5357 - val_loss: 0.5330 - val_accuracy: 0.7645 - val_f1_score: 0.3368
Epoch 5/10
4688/4688 [==============================] - 57s 12ms/step - loss: 0.4017 - accuracy: 0.8318 - f1_score: 0.5900 - val_loss: 0.5875 - val_accuracy: 0.7459 - val_f1_score: 0.4550
Epoch 6/10
4688/4688 [==============================] - 56s 

### Model 10: Dropout 10%

I am now decreasing the dropout to 10%, hoping it caused validation accuracy to rise even more compared to model 8. 

In [ ]:
model_8 = train_drop_model(0.1)

Epoch 1/10
4688/4688 [==============================] - 57s 12ms/step - loss: 0.5126 - accuracy: 0.7612 - f1_score: 0.2972 - val_loss: 0.5080 - val_accuracy: 0.7666 - val_f1_score: 0.3703
Epoch 2/10
4688/4688 [==============================] - 55s 12ms/step - loss: 0.4870 - accuracy: 0.7771 - f1_score: 0.3994 - val_loss: 0.4992 - val_accuracy: 0.7658 - val_f1_score: 0.4158
Epoch 3/10
4688/4688 [==============================] - 54s 12ms/step - loss: 0.4614 - accuracy: 0.7952 - f1_score: 0.4797 - val_loss: 0.5176 - val_accuracy: 0.7600 - val_f1_score: 0.4457
Epoch 4/10
4688/4688 [==============================] - 56s 12ms/step - loss: 0.4246 - accuracy: 0.8185 - f1_score: 0.5537 - val_loss: 0.5277 - val_accuracy: 0.7564 - val_f1_score: 0.4450
Epoch 5/10
4688/4688 [==============================] - 57s 12ms/step - loss: 0.3878 - accuracy: 0.8379 - f1_score: 0.6107 - val_loss: 0.5564 - val_accuracy: 0.7581 - val_f1_score: 0.3823
Epoch 6/10
4688/4688 [==============================] - 58s 

## 5.1: Training Final Model

In [ ]:
from tensorflow.keras import models
from tensorflow.keras import layers
from keras.layers import Embedding, Flatten

model = models.Sequential()
model.add(Embedding(15000, 8, input_length=maxlen))
model.add(Flatten())
model.add(layers.Dense(128,  activation = 'relu'))
model.add(layers.Dropout(0.1))
model.add(layers.Dense(64,  activation = 'relu'))
model.add(layers.Dropout(0.1))
model.add(layers.Dense(32, activation = 'relu'))
model.add(layers.Dense(1, activation = 'sigmoid'))

model.compile(optimizer = 'rmsprop', 
              loss = 'binary_crossentropy', 
              metrics = ['accuracy', f1_score])

model.fit(x_train, y_train, epochs = 2, batch_size = 32)

Epoch 1/2
6250/6250 [==============================] - 67s 11ms/step - loss: 0.5092 - accuracy: 0.7618 - f1_score: 0.3102
Epoch 2/2
6250/6250 [==============================] - 71s 11ms/step - loss: 0.4925 - accuracy: 0.7747 - f1_score: 0.3869


In [ ]:
results = model.evaluate(x_test, y_test)

3125/3125 [==============================] - 9s 3ms/step - loss: 0.5217 - accuracy: 0.7571 - f1_score: 0.4644


In [ ]:
y_pred = (model.predict(x_test) > 0.5).astype("int32")

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
print('Accuracy: %.2f' % (accuracy_score(y_test, y_pred)))
print('Precision score: %.2f' % (precision_score(y_test, y_pred)))
print('Recall score: %.2f' % (recall_score(y_test, y_pred)))
print('F1 score: %.2f' % (f1_score(y_test, y_pred)))

Accuracy: 0.76
Precision score: 0.55
Recall score: 0.43
F1 score: 0.48
